In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
mainURL = "https://www.w3schools.com/java/default.asp"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')

urls = []
pages = soup.find('div', {'id': 'leftmenuinnerinner'}).findAll(href=True)
for slug in pages:
    urls.append('https://www.w3schools.com/java/' + slug.get('href'))

mainHeadings = []
subHeadingsH2 = []
subHeadingsH3 = []
links = []
imageURLs = []
facts = []
definitions = []
explanations = []
codes = []
listItems = []
tableData = []
data = {}


In [3]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html.parser')

    subContent = soup.find('div', {"id": "main"})  # Page Sub Content
    if subContent is None:
        continue

    mainHeading = subContent.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    link = subContent.findAll('a')  # Page Links
    pageLinks = []
    if link is not None:
        for lin in link:
            if lin.get('href') is not None and not lin.get('href').startswith('http'):
                pageLinks.append('https://www.w3schools.com/java/' + lin.get('href'))
            else:
                pageLinks.append(lin.get('href'))

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images:
            pageImageURLs.append('https://www.w3schools.com/java/' + img.get('src'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text.replace('\n', '')
            if 10 <= len(slug) <= 79:
                pageFacts.append(slug)
            elif 80 <= len(slug) <= 119:
                pageDefinitions.append(slug)
            elif 120 <= len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('div', {"class": "w3-code"})  # Codes and Outputs
    pageCodes = []
    if codeContent is not None:
        for slug in codeContent:
            pageCodes.append(slug.text)

    tables = subContent.findAll('table')  # Tables
    pageTableData = []
    if tables is not None:
        for tb in tables:
            table = ""
            rowData = []
            tableRows = tb.findAll('tr')
            for row in tableRows:
                rowData.append(row.text[1:-1].replace('\n', '|||'))
            for row in rowData:
                table = table + str(row) + '&&&'
            pageTableData.append(table[:-3])

    allListUL = subContent.findAll('ul')  # List Items
    allListOL = subContent.findAll('ol')
    listUL = ""
    listOL = ""
    listItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            for lis in allListUL:
                listUL = listUL + str(lis.text[1:]).replace('\n', '|||').replace('\t\t', '')
        if len(allListOL) != 0:
            for lis in allListOL:
                listOL = listOL + str(lis.text[1:]).replace('\n', '|||').replace('\t\t', '')
        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + listUL
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + listOL
        else:
            listItem = listItem + listUL + '&&&' + listOL
        pageListItems.append(listItem)

    # print(pageMainHeadings)
    # print(pageSubHeadingsH2)
    # print(pageSubHeadingsH3)
    # print(pageLinks)
    # print(pageImageURLs)
    # print(pageFacts)
    # print(pageDefinitions)
    # print(pageExplanations)
    # print(pageCodes)
    # print(pageTableData)
    # print(pageListItems)

    mainHeadings.append(pageMainHeadings)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    links.append(pageLinks)
    imageURLs.append(pageImageURLs)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    codes.append(pageCodes)
    tableData.append(pageTableData)
    listItems.append(pageListItems)



In [4]:
data = {'main_headings': mainHeadings, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'links': links, 'image_urls': imageURLs, 'facts': facts, 'definitions': definitions,
        'explanations': explanations, 'codes': codes, 'table_data': tableData, 'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('W3Schools_Java.csv')

In [5]:
a=pd.read_csv("W3Schools_Java.csv")
a

,Unnamed: 0,main_headings,sub_headings_h2,sub_headings_h3,links,image_urls,facts,definitions,explanations,codes,table_data,list_items
0,0,['Java Tutorial'],"['Examples in Each Chapter', 'Java Exercises',...",['Example'],['https://www.w3schools.com/java//default.asp'...,[],"['Java is a popular programming language.', 'T...","['Java is used to develop mobile apps, web app...",['Java is an object oriented language and some...,[],[],[]
1,1,['Java Introduction'],"['What is Java?', 'Why Use Java?', 'Get Started']",[],"['https://www.w3schools.com/java/default.asp',...",[],"['Java is a popular programming language, crea...",[],[],[],[],['Mobile applications (specially Android apps)...
2,2,['Java Getting Started'],"['Java Install', 'Setup for Windows', 'Step 1'...",[],['https://www.w3schools.com/java/java_intro.as...,"['https://www.w3schools.com/java/system.png', ...",['Some PCs might have Java already installed.'...,['If you do not have Java installed on your co...,['To check if you have Java installed on a Win...,"['\nC:\\Users\\Your Name>java -version\n', '\n...",[],"['Go to ""System Properties"" (Can be found on C..."
3,3,['Java Syntax'],"['Java Syntax', 'The main Method', 'System.out...",['Example explained'],['https://www.w3schools.com/java/java_getstart...,[],"['Try it Yourself »', 'Note: Java is case-sens...","['Inside the main() method, we can use the pri...","['In the previous chapter, we created a Java f...",[],[],[]
4,4,['Java Comments'],"['Java Comments', 'Single-line Comments', 'Jav...","['Example', 'Example', 'Example']",['https://www.w3schools.com/java/java_syntax.a...,[],['Single-line comments start with two forward ...,['Any text between // and the end of the line ...,"['Comments can be used to explain Java code, a...",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
99,99,['Java Examples'],"['Java Syntax', 'Java Comments', 'Java Variabl...",[],['https://www.w3schools.com/java/java_ref_math...,[],[],[],[],[],[],[]
100,100,['Java Online Compiler'],"['Java Compiler (Editor)', 'Java Compiler Expl...",[],['https://www.w3schools.com/java/java_examples...,[],"['Click on the ""Try it Yourself"" button to see...","['With our online Java compiler, you can edit ...",['The window to the left is editable - edit th...,[],['Icon|||Description&&&|||Go to www.w3schools....,[]
101,101,['Java Exercises'],"['Exercises', 'Count Your Score', 'Start Java ...",[],['https://www.w3schools.com/java/java_compiler...,[],"[""You can test your Java skills with W3Schools...",['We have gathered a variety of Java exercises...,[],[],[],[]
102,102,['Java Quiz'],"['The Test', 'Count Your Score', 'Start the Qu...",[],['https://www.w3schools.com/java/java_exercise...,[],"[""You can test your Java skills with W3Schools...","[""The test is not official, it's just a nice w...",['You will get 1 point for each correct answer...,[],[],[]
